# Imports and basic set up

In [2]:
#In the begining, ther is time
import time

In [3]:
#Quality of life
#Some things we do generate a lot of warnings, and it just becomes clutter.
import warnings
warnings.filterwarnings("ignore")
#I like it when my notebook helps me out
%config IPCompleter.greedy=True
#Sometiems you just need to print pretty
from pprint import pprint
#Lots of these operations take many minutes to complete. So it behooves us to pickle the outputs and just unpickle them each time we re-open the notebook
import pickle


Data is from: https://www.kaggle.com/datasets/toygarr/datasets-for-natural-language-processing

Dataset is from a collection of sentiment datasets, but we just want to play with the food one for now.

"ctweet, stweet, food" datasets are positive or negative analysis (sentiment) -> 0 negative -> 1 positive (ctweet has neutral 0, 1, 2)

we're assuming that any code we write to deal with this food data set will be extensible later by simply adding the rest of the data if we so choose. At the moment it's simply faster to work with a smaller subset of the data as we design the pipeline and it's displays.

## Utilities

In [4]:
import pandas as pd
import nltk
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk import ngrams
import matplotlib.pyplot as plt

## Corpus and Vectoring tools

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import DBSCAN
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary
import os
from sklearn.model_selection import train_test_split


In [6]:
from sklearn.utils import resample

# Load the data

In [7]:
test = pd.read_csv('C:/Users/Prathmun/Documents/Springboard Jupyter/Sentiment-Capstone/data/food/test.csv')
train = pd.read_csv('C:/Users/Prathmun/Documents/Springboard Jupyter/Sentiment-Capstone/data/food/train.csv')
frames = [train, test]
df = pd.concat(frames)

Nobs to play with, want 'em nearby

In [8]:
sample_size = 50000
num_feats =10

# Sampling 

Here we can set up the code to downsample for speed's sake

In [9]:
#df = df.head(2000)

In [10]:
# sample_size = 50000
# df = resample(df,
#                          replace=False,
#                          n_samples=sample_size,
#                         random_state=12,
#                         )

In [11]:
df.shape

(363219, 2)

In [12]:
df.Y.value_counts()

1    306223
0     56996
Name: Y, dtype: int64

# Processing the text

### Tokenize text

In [13]:
df['tokenized'] = df.text.apply(word_tokenize)



In [14]:
df.head(2)

,text,Y,tokenized
0,i was put off at first by the green powder but...,1,"[i, was, put, off, at, first, by, the, green, ..."
1,these ginger chews are too good to be true i t...,1,"[these, ginger, chews, are, too, good, to, be,..."


### Remove Stop Words


In [15]:
stopwords = nltk.corpus.stopwords.words("english")

In [16]:
df['nostops'] = df.tokenized.apply(lambda  x: [item for item in x if item not in stopwords])

In [17]:
df.head(2)

,text,Y,tokenized,nostops
0,i was put off at first by the green powder but...,1,"[i, was, put, off, at, first, by, the, green, ...","[put, first, green, powder, bad, little, grain..."
1,these ginger chews are too good to be true i t...,1,"[these, ginger, chews, are, too, good, to, be,...","[ginger, chews, good, true, try, limit, one, d..."


### Stemming 
Might do lemmatization later, but stemming is simpler and more reliable and we're looking to get something working before we refine it. Lemmatization is a step we can experiment with in our refinement stages

We chose snowball because it's an older, and stable stemmer that incorporates improvements from the older stemmer algorithm Porter's real world experience
Snowball docs: https://www.nltk.org/api/nltk.stem.snowball.html
Article that informed our decision to choose Snowball: https://towardsdatascience.com/stemming-lemmatization-what-ba782b7c0bd8


It looks as if stemming can increase recall, even in short texts but can also cause problems. We're going to move forward with stemmed words for now, but again can return to this if we seek optimization tasks down the line
Source: https://stackoverflow.com/questions/47219389/compute-word-n-grams-on-original-text-or-after-lemma-stemming-process#:~:text=Computing%20word%20n%2Dgrams%20after,you%20want%20to%20do%20it.

In [18]:
stemmer = SnowballStemmer("english")
df['stemmed'] = df.nostops.apply(lambda x: [stemmer.stem(item) for item in x])

In [19]:
df.stemmed.head(2)

0    [put, first, green, powder, bad, littl, graini...
1    [ginger, chew, good, true, tri, limit, one, da...
Name: stemmed, dtype: object

# Building Limited Vocabularies

In [55]:
nostops_text_neg = " ".join(" ".join(listo) for listo in df.nostops[df['Y'] == 0])

In [56]:
neg_list = nostops_text_neg.split(' ')
neg_set = set(neg_list)

In [57]:
nostops_text_pos = " ".join(" ".join(listo) for listo in df.nostops[df['Y'] == 1])

In [58]:
pos_list = nostops_text_pos.split(' ')
pos_set = set(pos_list)

In [59]:
print('neg list len = ' + str(len(neg_list)))
print('neg set len = ' + str(len(neg_set))) 

neg list len = 2334177
neg set len = 46886


In [60]:
print('pos list len = ' + str(len(pos_list)))
print('pos set len = ' + str(len(pos_set))) 

pos list len = 11470406
pos set len = 104877


## Ratio DF

## Extracting Negative Only words

In [61]:
neg_only_set = neg_set - pos_set
len(neg_only_set )

11892

### Neg only Count DF

In [62]:
#Building up the count for the neg only df so that we can order them to slice, so we can add them to the lexicon we're going to feed into the vocabulary of the tokenizer

neg_count = {}

for word in neg_list:
    if word in neg_only_set:
        if word not in neg_count.keys():
            
            neg_count[word] = 0
            
        neg_count[word] += 1
        
neg_count_df = pd.DataFrame(neg_count, index=[0])
                            
neg_only_count_df = neg_count_df.transpose()

len(neg_only_set)

11892

## Extracting Positive Only Words

In [63]:
pos_only_set =  pos_set - neg_set 
len(pos_only_set )

69883

### Pos Only Count Df

In [64]:
#Building up the count for the neg only df so that we can order them to slice, so we can add them to the lexicon we're going to feed into the vocabulary of the tokenizer

pos_count = {}

for word in pos_list:
    if word in pos_only_set:
        if word not in pos_count.keys():
            
            pos_count[word] = 0
            
        pos_count[word] += 1
        
pos_count_df = pd.DataFrame(pos_count, index=[0])
                            
pos_only_count_df = pos_count_df.transpose()

len(pos_only_set)

69883

### Extracting word ratios

Using set logic to snag words that only appear in both positive and negative polarity documents

In [65]:
shared_set = neg_set.intersection(pos_set)
len(shared_set)

34994

In [66]:
ratio_df =  pd.DataFrame(shared_set)

### Counting the negative appearences of our set of words

In [67]:
neg_count = {}

for word in neg_list:
    if word in shared_set:
        if word not in neg_count.keys():
            
            neg_count[word] = 0
            
        neg_count[word] += 1
        
neg_count_df = pd.DataFrame(neg_count, index=[0])
                            
neg_count_df = neg_count_df.transpose()

### Counting the positive appearences of our set of words

In [68]:
pos_count = {}

for word in pos_list:
    if word in shared_set:
        if word not in pos_count.keys():
            
            pos_count[word] = 0
            
        pos_count[word] += 1

pos_count_df = pd.DataFrame(pos_count, index=[0])

pos_count_df = pos_count_df.transpose()


In [69]:
### Stapling the pos/neg counts to the ratio df

In [71]:
ratio_df['neg_count'] = neg_count_df[0]
ratio_df['pos_count'] = pos_count_df[0]

### Getting ratiod

In [72]:
ratio_df['ratio'] = ratio_df[['neg_count','pos_count']].mean(axis=1)

# Function to slice the lexicons appropriately

In [81]:
def slicer(pos_only, neg_only, high_ratio):
    
    #slice pos
    pos_slice = list(pos_only_count_df.sort_values(by=[0], ascending=False).head(pos_only).index)
    print(len(pos_slice))
    #slice neg
    neg_slice = list(neg_only_count_df.sort_values(by=[0], ascending=False).head(neg_only).index)
    print(len(neg_slice))
    #slice high_ratio
    ratio_slice = list(ratio_df.sort_values(ascending=False, by='ratio').head(high_ratio+1).index)[1:]
    print(len(ratio_slice))
    
    
    #Join slices into one big list
    sliced_lexicons = [pos_slice,neg_slice,ratio_slice]
    sliced_lexicon = []
    for lexicon in sliced_lexicons:
        for word in lexicon:
            sliced_lexicon.append(word)
        
    #Return list
    return sliced_lexicon

# <span style="color:purple"> Clustering and Secondary EDA/Feature Engineering</span>

We played briefly with using DBSCAN, but it didn't converge at all, it gave us 360k categories

Thinking that instead of conventional clustering we're going to play with latent dirichlet allocation because it'll do a similar thing as K-means, but also provide us with a lot more information about the composition of those clusters.
Original Paper: https://ai.stanford.edu/~ang/papers/nips01-lda.pdf
Tutorial we used to help us get going: https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0
Docs for the gensim module we're sdoing the lda with: https://radimrehurek.com/gensim/models/ldamodel.html

In [19]:
stemmed_texts = list(df.stemmed)

In [20]:
#makes a dictionary mapping to unique IDs
id2word = Dictionary(stemmed_texts)


In [21]:
#makes a bag of words version of the Gensim Dictionary
corpus = [id2word.doc2bow(text) for text in stemmed_texts]


In [40]:
# # Train the model on the corpus.
start = time.time()
num_topics = 150
lda = LdaModel(corpus,  id2word=id2word, num_topics=num_topics)
stop = time.time()
print("training the model took " +str(stop-start) + " seconds to complete")

this took 75.45151829719543 seconds to complete


# Feature Engineering with LDA

## Topic Vectors

Documentation we're going to need to use to get at document relationships to topics: https://radimrehurek.com/gensim/models/ldamodel.html

Relevant stack overflow
https://stackoverflow.com/questions/43357247/get-document-topics-and-get-term-topics-in-gensim

In [41]:
import gensim

In [42]:
start = time.time()
#Stole the below code from
#https://stackoverflow.com/questions/46574720/python-gensim-lda-add-the-topic-to-the-document-after-getting-the-topics
all_topics = lda.get_document_topics(corpus, minimum_probability=0.0)
all_topics_csr = gensim.matutils.corpus2csc(all_topics)
all_topics_numpy = all_topics_csr.T.toarray()
all_topics_df = pd.DataFrame(all_topics_numpy)
stop = time.time()
print("pulling the topcs took " +str(stop-start) + " seconds to complete")

this took 58.73469805717468 seconds to complete


In [43]:
#This is a feature of designing our code with downsampled data. Just need the index's to be consistent.
all_topics_df.index = df.index

In [44]:
#Let's rename the topic columns so that we can identify them later on
for each in all_topics_df.columns:
    all_topics_df['topic_' + str(each)] = all_topics_df[each]
    all_topics_df.drop(columns=[each], inplace=True)

In [36]:
all_topics_df

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,...,topic_110,topic_111,topic_112,topic_113,topic_114,topic_115,topic_116,topic_117,topic_118,topic_119
33425,0.000269,0.000269,0.000269,0.000269,0.000269,0.000269,0.000269,0.000269,0.000269,0.000269,...,0.000269,0.000269,0.000269,0.000269,0.000269,0.000269,0.000269,0.000269,0.000269,0.000269
174759,0.000309,0.000309,0.000309,0.000309,0.000309,0.000309,0.000309,0.000309,0.000309,0.000309,...,0.000309,0.000309,0.000309,0.000309,0.000309,0.000309,0.000309,0.000309,0.000309,0.000309
111229,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333,...,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333,0.048490,0.000333
209101,0.000397,0.313755,0.000397,0.000397,0.000397,0.000397,0.000397,0.000397,0.000397,0.000397,...,0.000397,0.000397,0.000397,0.000397,0.000397,0.000397,0.000397,0.000397,0.000397,0.000397
109643,0.145762,0.000758,0.000758,0.000758,0.000758,0.000758,0.000758,0.000758,0.000758,0.106666,...,0.000758,0.000758,0.000758,0.000758,0.000758,0.000758,0.000758,0.000758,0.135033,0.000758
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59447,0.000096,0.000096,0.000096,0.000096,0.069769,0.000096,0.169432,0.000096,0.029456,0.000096,...,0.000096,0.000096,0.000096,0.000096,0.024203,0.000096,0.000096,0.018131,0.012412,0.000096
97003,0.089298,0.000362,0.000362,0.000362,0.000362,0.000362,0.223862,0.000362,0.000362,0.000362,...,0.000362,0.000362,0.000362,0.000362,0.000362,0.053188,0.000362,0.000362,0.000362,0.000362
64465,0.000181,0.000181,0.069206,0.026357,0.000181,0.023418,0.000181,0.000181,0.065573,0.000181,...,0.000181,0.029271,0.000181,0.000181,0.000181,0.000181,0.000181,0.057708,0.000181,0.000181
91943,0.011404,0.000023,0.000023,0.015598,0.041362,0.000023,0.007600,0.016370,0.004398,0.006995,...,0.088138,0.000023,0.016547,0.012009,0.012541,0.013296,0.000023,0.030237,0.008304,0.000023


In [37]:
import scipy

In [48]:
def topic_model_and_dump(num_topics=100):

    print('Running ' + str(num_topics) + ' topic LDA')
    
    # # Train the model on the corpus.
    start = time.time()
    lda = LdaModel(corpus,  id2word=id2word, num_topics=num_topics)
    stop = time.time()
    print("training the model took " +str(stop-start) + " seconds to complete")

    start = time.time()
    #Stole the below code from
    #https://stackoverflow.com/questions/46574720/python-gensim-lda-add-the-topic-to-the-document-after-getting-the-topics
    all_topics = lda.get_document_topics(corpus, minimum_probability=0.0)
    all_topics_csr = gensim.matutils.corpus2csc(all_topics)
    all_topics_numpy = all_topics_csr.T.toarray()
    all_topics_df = pd.DataFrame(all_topics_numpy)
    stop = time.time()
    print("pulling the topcs took " +str(stop-start) + " seconds to complete")

    #Let's rename the topic columns so that we can identify them later on
    all_topics_df.index = df.index
    for each in all_topics_df.columns:
        all_topics_df['topic_' + str(each)] = all_topics_df[each]
        all_topics_df.drop(columns=[each], inplace=True)

    start = time.time()
    return all_topics_df    
    #pickle.dump(all_topics_df, open("full_df_" +str(num_topics) + "_topics", 'wb'))
    stop = time.time()
    print('Succesfully saved ' + str(num_topics) + ' dataframe' + 'it took ' +str(stop-start) + ' seoconds to save')


# Vectorizing the corpus
#Creating a df for each degree of n-gram vector, for easy organization
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

Because of the limited dimensionality we have to play with we've opted to build our own vocabulary for the tokenizer.

In [82]:
lexicon = slicer(50, 50, 100)

50
50
100


In [83]:
len(lexicon)

200

In [84]:
Vectorizer_Agent_mono = TfidfVectorizer(preprocessor=None, stop_words=stopwords, max_features=num_feats)

In [85]:
vectored_no_stops_mono = Vectorizer_Agent_mono.fit_transform(df.text.values) 

In [86]:
vectored_no_stops_mono.shape

(363219, 10)

In [87]:
pos_only_list= [0,0,100,50,50,25]
neg_only_list= [0,0,100,50,50,50]
high_ratio_list = [0,100,0,100,200,225]
topic_list= [600,500,400,400,300,300]

In [90]:
grid_space = {'pos_only' : pos_only_list,
             'neg_only':neg_only_list,
             'high_ratio':high_ratio_list,
            'topics':topic_list}

# Aggregating our X

In [158]:
mono_and_topics = [vectored_no_stops_mono, topic_vector]

In [159]:
X = hstack(mono_and_topics)

## Bringing in and extracting our Y column 

In [115]:
test = pd.read_csv('C:/Users/Prathmun/Documents/Springboard Jupyter/Sentiment-Capstone/data/food/test.csv')
train = pd.read_csv('C:/Users/Prathmun/Documents/Springboard Jupyter/Sentiment-Capstone/data/food/train.csv')
frames = [train, test]
df = pd.concat(frames)


In [13]:
df = resample(df,
                         replace=False,
                         n_samples=int(sample_size),
                        random_state=12,
                        )

In [161]:
y = df.Y

In [24]:
y.value_counts(normalized=True)

NameError: name 'y' is not defined

# Train Test Split

In [162]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [163]:
print(X_train.shape)
print(y_train.shape)

(37500, 10)
(37500,)


# Save our Xs and ys to pass onto the next notebook

In [164]:
pickle.dump(X_train, open("X_train", 'wb'))
pickle.dump(X_test, open("X_test", 'wb'))
pickle.dump(y_train, open("y_train", 'wb'))
pickle.dump(y_test, open("y_test", 'wb'))

# Process to map modeling times based on dimensionality

In [52]:
def timing_buddy(model_to_fit, dictionary_string, dicto):
    funky_time_start = time.time()
    model_to_fit.fit(X_train, np.ravel(y_train))
    funky_time_stop = time.time()
    funky_train_time = funky_time_stop - funky_time_start
    dicto[dictionary_string]['training_time'] = funky_train_time
    

In [53]:
def confused_buddy(model_to_confuse, dictionary_string,dicto):
    confuse = confusion_matrix(y_test, model_to_confuse.predict(X_test))
    dicto[dictionary_string]['confuse'] = confuse

In [54]:
def recall_calculator(confuse):
    recall = 0
    tp = confuse[1][1]
    fn = confuse[1][0]
    if tp > 0 or tp ==1:
        recall = tp / (tp+fn)
    return recall

In [92]:
def bayes_hyper_grad_boosting_model(dicto, X_train, y_train, X_test, y_test):
    
    def GradBoost_hyper(n_estimators, learning_rate, max_leaf_nodes):
        grady_the_boosted = GradientBoostingClassifier(learning_rate=learning_rate,  n_estimators=int(n_estimators),
                                                      max_leaf_nodes=int(max_leaf_nodes))
        grady_the_boosted.fit(X_train, np.ravel(y_train))
        confuse = confusion_matrix(y_test, grady_the_boosted.predict(X_test))
        recall = recall_calculator(confuse)
        return recall
    
    ###
    GradBoost_params = {'learning_rate' : (.1,2),
                    'n_estimators':(10,500),
                    'max_leaf_nodes':(3,50)}
#     ###
    
#     optimizer = BayesianOptimization(
#     GradBoost_hyper,
#     pbounds=GradBoost_params,
#     verbose=1)
    
#     ######
    
#     print('Gradient Boosting')
#     optimizer.maximize(init_points=3, n_iter=4)
#     grad_boost_params= optimizer.max
#     grad_boost_params = grad_boost_params['params']
    
    ###
    grady_the_boosted = GradientBoostingClassifier(learning_rate=1.0 ,
                                                   n_estimators=100 ,
                                                   max_leaf_nodes=4 )
    timing_buddy(grady_the_boosted, 'GradientBoostingClassifier', dicto)
    confused_buddy(grady_the_boosted, 'GradientBoostingClassifier', dicto)
    dicto['GradientBoostingClassifier']['ROC_AUC_Score'] = roc_auc_score(y_test, grady_the_boosted.predict_proba(X_test)[:, 1])
    dicto['GradientBoostingClassifier']['Hyper_Params'] = grad_boost_params
    

In [ ]:
def grid_maker(grid_dict)

for i in range[len(grid_dict['pos_only'])]
    num_topcs = grid_dict['topics'][i]    
    all_topics_df = topic_model_and_dump(num_topics)
        
    topic_vector = scipy.sparse.csr_matrix(all_topics_df.values)
    
    neg_only = grid_dict['neg_only'][i]
    high_ratio = grid_dict['high_ratio'][i]
    lexicon = slicer(pos_only, neg_only, high_ratio)

    Vectorizer_Agent_mono = TfidfVectorizer(preprocessor=None, stop_words=stopwords, max_features=num_feats, vocabulary=lexicon)

    mono_and_topics = [vectored_no_stops_mono, topic_vector]
    
    X = hstack(mono_and_topics)
    y= df.Y
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    
    
    name +'_pos_' + str(pos_only) +'_neg_' + str('neg_only') + '_ratio_'


In [94]:
def chonky_model_aggregator(X_train, y_train, X_test, y_test):
    
    #Models to explore
    model_set = ['GradientBoostingClassifier']
    model_stats = {}
    for each in model_set:
        model_stats[each] ={'confuse' : [], 'training_time' : 0, 'ROC_AUC_Score': 0, 'Hyper_Params': {}}
    bayes_hyper_grad_boosting_model(model_stats, X_train, y_train, X_test, y_test)
    

    return model_stats



In [60]:
import pandas as pd

In [61]:
accumulator = pd.DataFrame(columns=['confuse', 'training_time', 'hyper_params', 'ROC_AUC_Score'])

In [62]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, roc_auc_score

from bayes_opt import BayesianOptimization

In [63]:
accumulator

,confuse,training_time,hyper_params,ROC_AUC_Score


In [64]:
test = {4:5,3:2}

In [65]:
test

{4: 5, 3: 2}

In [66]:
len(test)

2

In [101]:
y = df.Y

In [102]:
y.shape

(363219,)

In [76]:


def dimensional_mapper(features):
    details_dict = {}
    time_to_execute= 0

    #Make the X vector with the proper number of features
        ##y should stay the same, so we're not rebuilding it each iteration

    Vectorizer_Agent_mono = TfidfVectorizer(preprocessor=None, stop_words=stopwords, max_features=int(features))
    X = Vectorizer_Agent_mono.fit_transform(df.text.values) 
    print(X.shape)

    #train/test split it

    X_train, X_test, y_train, y_test = train_test_split(X, y)

    #run the model

    stats = chonky_model_aggregator(X_train, y_train, X_test, y_test)

    #record it's stats

    time_to_execute = stats['GradientBoostingClassifier']['training_time']
    details_dict[features] = stats['GradientBoostingClassifier']

    #increase the number of features
    print('the last iteration took ' + str(time_to_execute) + ' seconds to train')
    print('FEATURES = ' + str(features))
    print(' ')
    print(' ')
    print(' ')
    if time_to_execute < 600:
        return -600 + time_to_execute
    if time_to_execture > 600:
        return 600 - time_to_execute
    if time_to_execute == 600:
        return 100
    
    

In [77]:
dimensional_params = {'features': (10,5000)}


In [78]:
dimensional_optimizer = BayesianOptimization(
    dimensional_mapper,
    pbounds=dimensional_params,
    verbose=1)

In [79]:
dimensional_optimizer.maximize(init_points=4, n_iter=20)
grad_boost_params= optimizer.min
grad_boost_params = grad_boost_params['params']

|   iter    |  target   | features  |
-------------------------------------
(50000, 2029)
Gradient Boosting
|   iter    |  target   | learni... | max_le... | n_esti... |
-------------------------------------------------------------
|  3        |  0.9699   |  1.675    |  21.58    |  482.5    |
|  6        |  0.97     |  1.841    |  18.69    |  164.3    |
|  8        |  0.9771   |  1.818    |  4.346    |  276.0    |
|  15       |  0.9791   |  1.372    |  27.02    |  99.05    |


ValueError: X has 2029 features, but DecisionTreeRegressor is expecting 44 features as input.

In [111]:
features = 700
time_to_execute= 0
details_dict = {}

while 600 > time_to_execute:
    
    #Make the X vector with the proper number of features
        ##y should stay the same, so we're not rebuilding it each iteration
        
    Vectorizer_Agent_mono = TfidfVectorizer(preprocessor=None, stop_words=stopwords, max_features=features)
    X = Vectorizer_Agent_mono.fit_transform(df.text.values) 
    print(X.shape)
    
    #train/test split it
    
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    
    #run the model
    
    stats = chonky_model_aggregator(X_train, y_train, X_test, y_test)
    
    #record it's stats
    
    time_to_execute = stats['GradientBoostingClassifier']['training_time']
    details_dict[features] = stats['GradientBoostingClassifier']
    
    #increase the number of features
    print('the last iteration took ' + str(time_to_execute) + ' seconds to train')
    print('FEATURES = ' + str(features))
    print(' ')
    print(' ')
    print(' ')
    features += 100
    if len(details_dict) > 4:
        pickle.dump(details_dict, open("dimensionality_training_data_max_" +str(features) + '_features', 'wb'))
        details_dict = {}

    
    

(363219, 700)
Gradient Boosting
|   iter    |  target   | learni... | max_le... | n_esti... |
-------------------------------------------------------------
|  5        |  0.967    |  0.5597   |  34.93    |  378.3    |
|  6        |  0.9852   |  1.9      |  4.72     |  98.7     |
|  7        |  0.9875   |  1.905    |  26.04    |  267.5    |
the last iteration took 768.8696720600128 seconds to train
FEATURES = 700
 
 
 


In [112]:
details_dict

{700: {'confuse': array([[  826, 13523],
         [  959, 75497]], dtype=int64),
  'training_time': 768.8696720600128,
  'ROC_AUC_Score': 0.579698770470151,
  'Hyper_Params': {'learning_rate': 1.9045136766473334,
   'max_leaf_nodes': 26.036159694472282,
   'n_estimators': 267.48774390256347}}}

In [107]:
details_df = pd.DataFrame(details_dict)
details_df.head(2)

,600
Hyper_Params,"{'learning_rate': 1.8157960344591315, 'max_lea..."
ROC_AUC_Score,0.527477


In [109]:
details_df.training_time.plot()

AttributeError: 'DataFrame' object has no attribute 'training_time'

In [110]:
pickle.dump(details_df, open("modeling_training_curve_700", 'wb'))
